## Apprentissage en contexte

Ce notebook implémente et compare différentes approches pour réaliser la tâche de NLI en utilisant directement un LLM pré-entraîné (Llama 3.2 3B Instruct) avec différentes stratégies de prompting.
**Expériences :**
- **0-shot :** Demander au modèle de classer sans exemples.
- **Few-shot :** Fournir quelques exemples de prémisse/hypothèse/étiquette avant de demander la prédiction.
- **Chain-of-Thought (CoT) :** Prompter le modèle pour qu'il explique pourquoi il a choisi une étiquette. Cela répond à l'objectif pédagogique de compréhension du CoT.

### Installation des dépendances

In [1]:
!pip install -U -q transformers accelerate bitsandbytes kagglehub
## !pip install torch numpy pandas polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.2 MB/s eta 0:00:00


### Chargement des données

In [2]:
import polars as pl

df_train = pl.read_csv(
    "https://raw.githubusercontent.com/mathisdrn/nlp-representation-learning/refs/heads/main/data/nli_fr_train.tsv",
    separator="\t",
    new_columns=["premise", "hypothesis", "label"],
)
df_val = pl.read_csv(
    "https://raw.githubusercontent.com/mathisdrn/nlp-representation-learning/refs/heads/main/data/nli_fr_test.tsv",
    separator="\t",
    new_columns=["premise", "hypothesis", "label"],
)
label_mapping = {
    "neutral": "Neutre",
    "entailment": "Conséquence",
    "contradiction": "Contradiction",
}
df_train = df_train.with_columns(pl.col("label").replace(label_mapping))
df_val = df_val.with_columns(pl.col("label").replace(label_mapping))
df_train, df_val

(shape: (5_010, 3)
 ┌─────────────────────────────────┬─────────────────────────────────┬───────────────┐
 │ premise                         ┆ hypothesis                      ┆ label         │
 │ ---                             ┆ ---                             ┆ ---           │
 │ str                             ┆ str                             ┆ str           │
 ╞═════════════════════════════════╪═════════════════════════════════╪═══════════════╡
 │ Eh bien, je ne pensais même pa… ┆ Je ne lui ai pas parlé de nouv… ┆ Contradiction │
 │ Eh bien, je ne pensais même pa… ┆ J'étais si contrarié que je co… ┆ Conséquence   │
 │ Eh bien, je ne pensais même pa… ┆ Nous avons eu une grande discu… ┆ Neutre        │
 │ Et je pensais que c'était un p… ┆ Je n'avais pas conscience que … ┆ Neutre        │
 │ Et je pensais que c'était un p… ┆ J'avais l'impression que j'éta… ┆ Conséquence   │
 │ …                               ┆ …                               ┆ …             │
 │ Davidson ne devrait p

### Définition des prompts

In [ ]:
zero_shot_prompt = (
    "Tu es un expert en logique. "
    "Classifie la relation entre la prémisse et l'hypothèse en utilisant uniquement : "
    "Conséquence, Contradiction, ou Neutre.\n\n"
    "Classifie l'exemple suivant :\n"
)

# Define the template structure
example_template = pl.format(
    "Prémisse : {}\nHypothèse : {}\nClassification : {}\n---",
    pl.col("premise"),
    pl.col("hypothesis"),
    pl.col("label"),
)

# Extract a random example of each label
examples_str = (
    df_train.group_by("label")
    .tail(1)
    .select(example_template)
    .to_series()
    .str.join("\n")
    .item()
)

few_shot_prompt = (
    "Tu es un expert en logique. "
    + "Classifie la relation entre la prémisse et l'hypothèse en utilisant uniquement : "
    + "Conséquence, Contradiction, ou Neutre.\n\n"
    + "Voici des exemples de classification corrects :\n"
    + examples_str
    + "Réponds maintenant pour l'exemple suivant :\n"
)

cot_prompt = (
    "Tu es un expert en logique. "
    + "Classifie la relation entre la prémisse et l'hypothèse en utilisant uniquement : "
    + "Conséquence, Contradiction, ou Neutre.\n\n"
    + "Voici des exemples de classification corrects :\n"
    + examples_str
    + "\n Instructions pour la nouvelle tâche :\n"
    + "1. Analysez d'abord la logique étape par étape.\n"
    + "2. Conclus ensuite uniquement par le nom de la relation: Conséquence, Contradiction, ou Neutre.\n"
    + "Réponds maintenant pour l'exemple suivant :\n"
)
print(cot_prompt)

Tu es un expert en logique. Classifie la relation entre la prémisse et l'hypothèse en utilisant uniquement : Conséquence, Contradiction, ou Neutre.

Voici des exemples de classification corrects :
Prémisse : Le roman moyen de 200 000 mots pour 25 $ fonctionne à 8 000 mots par dollar.
Hypothèse : Un roman de 200 000 payé 25 $ revient à 4 000 mots par dollar.
Classification : Contradiction
---
Prémisse : Le roman moyen de 200 000 mots pour 25 $ fonctionne à 8 000 mots par dollar.
Hypothèse : Un roman de 200 000 mots à 25 $ est un prix équitable.
Classification : Neutre
---
Prémisse : Le roman moyen de 200 000 mots pour 25 $ fonctionne à 8 000 mots par dollar.
Hypothèse : Un roman de 200 000 mots pour 25 $, ça revient à 8 000 mots par dollar.
Classification : Conséquence
---
 Instructions pour la nouvelle tâche :
1. Analysez d'abord la logique étape par étape.
2. Conclus ensuite uniquement par le nom de la relation: Conséquence, Contradiction, ou Neutre.
Réponds maintenant pour l'exemple 

### Chargement du modèle (LLaMa 3.2 3B Instruct)

In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("HUGGING_FACE_HUB_TOKEN")
login(token=token)

In [5]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb

print(bnb.__version__)

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device: ", DEVICE)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, quantization_config=bnb_config, dtype=DTYPE, device_map="auto"
)
model = torch.compile(model)

0.49.0
Using device:  cuda


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

### Fonction d'inférence

In [ ]:
def predict_batch(
    premise: list[str], hypotheses: list[str], system_prompt: str, batch_size=32
) -> list[str]:
    """
    Génère les prédictions à partir des prémises et hypothèses.
    """
    # Formatage des prompts
    prompts = [
        tokenizer.apply_chat_template(
            [
                {
                    "role": "user",
                    "content": f"{system_prompt}\nPrémisse : {p}\nHypothèse : {h}\nClassification :",
                }
            ],
            tokenize=False,
            add_generation_prompt=True,
        )
        for p, h in zip(premise, hypotheses)
    ]

    results = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i : i + batch_size]

        inputs = tokenizer(
            batch, return_tensors="pt", padding=True, truncation=True, max_length=1024
        ).to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                use_cache=True,
            )

        # On garde uniquement les derniers tokens pour éviter de décoder le prompt à nouveau
        generated_ids = outputs[:, inputs.input_ids.shape[1] :]
        results.extend(tokenizer.batch_decode(generated_ids, skip_special_tokens=True))

    return results


def parse_response(response_col: str) -> pl.Expr:
    # Expression Regex pour trouver le label (insensible à la casse)
    label_pattern = r"(?i)\b(Conséquence|Contradiction|Neutre)\b"

    return (
        pl.col(response_col)
        # 1. Trouver toutes les labels dans une listes
        .str.extract_all(label_pattern)
        # 2. Garder le dernier labels
        .list.last()
        .str.to_titlecase()
        .fill_null("Prédiction incorrecte")
    )

### Jeu d'évaluation

In [8]:
sample_size = df_val.height
df_val = df_val.head(sample_size)

print(f"Taille de l'ensemble d'évaluation: {df_val.height} éléments")
print("Répartition des classes dans l'ensemble d'évaluation:")
print(df_val.get_column("label").value_counts(sort=True))

Taille de l'ensemble d'évaluation: 2490 éléments
Répartition des classes dans l'ensemble d'évaluation:
shape: (3, 2)
┌───────────────┬───────┐
│ label         ┆ count │
│ ---           ┆ ---   │
│ str           ┆ u32   │
╞═══════════════╪═══════╡
│ Neutre        ┆ 830   │
│ Contradiction ┆ 830   │
│ Conséquence   ┆ 830   │
└───────────────┴───────┘


In [9]:
premises = df_val.get_column("premise").to_list()
hypotheses = df_val.get_column("hypothesis").to_list()
valid_labels = ["Neutre", "Conséquence", "Contradiction"]

### Zero-shot learning

In [ ]:
# Générer les prédictions
predictions = predict_batch(premises, hypotheses, zero_shot_prompt)

# Ajout et parsing des prédictions
df_val = df_val.with_columns(predictions_zero_shot=pl.Series(predictions)).with_columns(
    parse_response("predictions_zero_shot")
)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# Prédictions non valides
valid = pl.col("predictions_zero_shot").is_in(valid_labels)

print(
    f"Nombre de prédictions invalides: ",
    df_val.filter(~valid).height,
    "/",
    df_val.height,
)
df_val.filter(~valid)

Nombre de prédictions invalides:  4 / 2490


premise,hypothesis,label,predictions_zero_shot
str,str,str,str
"""Au niveau inférieur, le direct…","""Le directeur ne voulait pas s'…","""Neutre""","""Prédiction incorrecte"""
"""Au niveau inférieur, le direct…","""Le directeur du service n'a pa…","""Conséquence""","""Prédiction incorrecte"""
"""Au niveau inférieur, le direct…","""Le directeur pensait que tout …","""Contradiction""","""Prédiction incorrecte"""
"""La Féminisation de la Culture …","""Il n'y a qu'une seule réponse …","""Neutre""","""Prédiction incorrecte"""


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

print(
    classification_report(
        df_val.get_column("label"),
        df_val.get_column("predictions_zero_shot"),
        zero_division=np.nan,
    )
)

                       precision    recall  f1-score   support

          Conséquence       0.49      0.09      0.15       830
        Contradiction       0.35      0.98      0.52       830
               Neutre       0.44      0.01      0.02       830
Prédiction incorrecte       0.00       nan      0.00         0

             accuracy                           0.36      2490
            macro avg       0.32      0.36      0.17      2490
         weighted avg       0.43      0.36      0.23      2490



### Few shots learning

In [ ]:
# Générer les prédictions
predictions = predict_batch(premises, hypotheses, few_shot_prompt)

# Ajout et parsing des prédictions
df_val = df_val.with_columns(predictions_few_shot=pl.Series(predictions)).with_columns(
    parse_response("predictions_few_shot")
)

In [ ]:
# Prédictions non valides
valid = pl.col("predictions_few_shot").is_in(valid_labels)

print(
    f"Nombre de prédictions invalides: ",
    df_val.filter(~valid).height,
    "/",
    df_val.height,
)
df_val.filter(~valid)

Nombre de prédictions invalides:  6 / 2490


premise,hypothesis,label,predictions_zero_shot,predictions_few_shot
str,str,str,str,str
"""Et si la médiation statistique…","""Il existe un théorie qui dit q…","""Neutre""","""Contradiction""","""Prédiction incorrecte"""
"""Tu as le culot de me réprimand…","""Il a choisi de ne pas prendre …","""Conséquence""","""Contradiction""","""Prédiction incorrecte"""
"""Tu as le culot de me réprimand…","""Il a choisi de lui prendre les…","""Contradiction""","""Contradiction""","""Prédiction incorrecte"""
"""Rami Yousef et Khalid Sheikh M…","""Khalid Sheikh Mohammed était b…","""Contradiction""","""Contradiction""","""Prédiction incorrecte"""
"""Bien que la xérographie ait pe…","""Xérographie est un mot apparen…","""Neutre""","""Conséquence""","""Prédiction incorrecte"""
"""faire du mal (à quelqu'un) - …","""Quelqu'un a été violé.""","""Neutre""","""Contradiction""","""Prédiction incorrecte"""


In [ ]:
print(
    classification_report(
        df_val.get_column("label"),
        df_val.get_column("predictions_few_shot"),
        zero_division=np.nan,
    )
)

                       precision    recall  f1-score   support

          Conséquence       0.34      0.97      0.51       830
        Contradiction       0.64      0.09      0.16       830
               Neutre       0.14      0.00      0.00       830
Prédiction incorrecte       0.00       nan      0.00         0

             accuracy                           0.36      2490
            macro avg       0.28      0.36      0.17      2490
         weighted avg       0.38      0.36      0.22      2490



### Chain-of-thoughts (CoT)

In [ ]:
# Générer les prédictions
predictions = predict_batch(premises, hypotheses, cot_prompt)

# Ajout et parsing des prédictions
df_val = df_val.with_columns(
    predictions_cot_raisonnement=pl.Series(predictions)
).with_columns(predictions_cot=parse_response("predictions_cot_raisonnement"))

In [17]:
# Prédictions non valides
valid = pl.col("predictions_cot").is_in(valid_labels)
invalid_examples = df_val.filter(~valid)

print(f"Nombre de prédictions invalides: ", invalid_examples.height, "/", df_val.height)
print("Exemple de prédictions invalides:\n")
for row in invalid_examples.sample(2).to_dicts():
    print("-" * 50)
    print(f"**Premise**: {row['premise']}")
    print(f"**Hypothesis**: {row['hypothesis']}")
    print(f"**Label**: {row['label']}")
    print(f"\n**Reasoning (CoT)**:\n{row['predictions_cot_raisonnement']}")
    print(f"\n**Final Prediction**: {row['predictions_cot']}")
    print("-" * 50 + "\n")

Nombre de prédictions invalides:  37 / 2490
Exemple de prédictions invalides:

--------------------------------------------------
**Premise**: Mais tout à coup, nous avons été appelés à regarder ce qui volait.
**Hypothesis**: Nous étions censés regarder ce qui était en train de voler.
**Label**: Conséquence

**Reasoning (CoT)**:
Analyse étape par étape :

1. La prémisse est "Mais tout à coup, nous avons été appelés à regarder ce qui volait."
   - Cela suggère un changement de situation ou d'attention, mais ce n'est pas une affirmation directe sur le vol.

2. La hypothèse est "Nous étions censés regarder ce qui était en train de voler."
   - Cela suggère une action précédente qui n'a pas été exécutée, mais ce n'est pas une affirmation directe sur le vol.

Analyse de la relation :

- La prémisse et l'hypothèse sont liées par un changement de situation ou d'attention, mais elles ne sont pas directement liées par une cause et un effet clair.
- La prémisse ne contredit pas l'hypothèse, car 

In [18]:
# Exemple de raisonnement sans parsing possible
df_val.filter(~valid).get_column("predictions_cot_raisonnement").item(0)

'Analyse étape par étape :\n\n1. La prémisse commence par une affirmation ("Je ne savais pas dans quoi je me lançais") qui est suivi d\'une condition ("donc j\'allais être rattaché à un lieu désigné à Washington"). Cela implique une cause (je ne savais pas) et un effet (j\'allais être rattaché à un lieu).\n2. La hypothèse commence par une condition ("Je n\'étais pas tout à fait certain de ce que j\'allais faire") qui est suivi d\'un résultat ("alors je suis allé à Washington où j\'étais chargé de faire un rapport"). Cela implique une cause (je n\'étais pas tout à fait certain) et un effet (je suis allé à Washington où j\'étais chargé de faire un rapport).\n\nEn comparant les deux, on remarque que la prémisse implique une cause (je ne savais pas) qui conduit à un effet (j\'allais être rattaché à un lieu), tandis que l\'hypothèse implique une condition (je n\'étais pas tout à fait certain) qui conduit à un effet ('

In [ ]:
print(
    classification_report(
        df_val.get_column("label"),
        df_val.get_column("predictions_cot"),
        zero_division=np.nan,
    )
)

                       precision    recall  f1-score   support

          Conséquence       0.36      0.89      0.52       830
        Contradiction       0.71      0.28      0.40       830
               Neutre       0.27      0.03      0.05       830
Prédiction incorrecte       0.00       nan      0.00         0

             accuracy                           0.40      2490
            macro avg       0.34      0.40      0.24      2490
         weighted avg       0.45      0.40      0.32      2490



Une faible proportions des réponses du modèle de langage ne sont pas transformable en prédictions pour diverses raisons :
- le modèle se répète à l'infini.
- le modèle refuse de répondre.
- le contexte maximum est atteint avant la fin de reflexion du modèle (limité pour des raisons de temps de calculs).

### Échantillons des prédictions

In [20]:
df_val.sample(10)

premise,hypothesis,label,predictions_zero_shot,predictions_few_shot,predictions_cot_raisonnement,predictions_cot
str,str,str,str,str,str,str
"""Peu après Boot vous trouverez …","""Le poste est près de La'al Rat…","""Conséquence""","""Contradiction""","""Conséquence""","""Analyse étape par étape : * L…","""Conséquence"""
"""À côté, une église octogonale …","""Le marché représente la renais…","""Contradiction""","""Contradiction""","""Conséquence""","""Analyse étape par étape : 1. …","""Conséquence"""
"""Comme Ross, Mehta se démène po…","""Ross et Mehta n'ont aucun prob…","""Contradiction""","""Contradiction""","""Conséquence""","""Analyse étape par étape : 1. …","""Contradiction"""
"""je ne sais pas d'accord, il ét…","""Je pense que je sais pourquoi …","""Neutre""","""Contradiction""","""Conséquence""","""Analyse étape par étape : 1. …","""Contradiction"""
"""La responsabilité est passée a…","""Le bureau de New York s'en est…","""Conséquence""","""Conséquence""","""Conséquence""","""Conséquence.""","""Conséquence"""
"""Cela peut être notre tour de c…","""Quelqu'un est en train de parl…","""Conséquence""","""Contradiction""","""Conséquence""","""Conséquence.""","""Conséquence"""
"""Le premier occidental à attein…","""James Cook n'est jamais allé à…","""Contradiction""","""Contradiction""","""Conséquence""","""Conséquence.""","""Conséquence"""
"""oui parce que c'est euh... ell…","""Ouais cela s'est fait sans une…","""Contradiction""","""Contradiction""","""Contradiction""","""Neutre""","""Neutre"""
"""Le gouvernement américain four…","""Le gouvernement Américain cons…","""Neutre""","""Contradiction""","""Conséquence""","""Conséquence.""","""Conséquence"""


### Analyse des Performances
L'examen des rapports de classification révèle des comportements distincts selon la méthode de sollicitation (prompting) utilisée :

| Méthode | Accuracy | Observations Clés |
|---------|----------|-------------------|
| Zero-shot | 36% | Biais massif vers la classe Contradiction (Recall de 0.98). Le modèle peine à identifier les Neutres et les Conséquences. |
| Few-shots | 36% | Le biais s'inverse radicalement vers la classe Conséquence (Recall de 0.97). La précision globale n'augmente pas. |
| Chain of Thought | 40% | Meilleur équilibre général. Amélioration notable du F1-score pour la Contradiction et légère hausse pour le Neutre. |

### Conclusion Technique
L'évaluation du modèle LLAMA 3.2 3B sur cette tâche de NLI en français permet de tirer les conclusions suivantes :

#### 1. Sensibilité extrême au formatage (Prompt Sensitivity)
Le modèle est très instable. En Zero-shot, il "prédit par défaut" la contradiction, tandis qu'en Few-shots, il bascule presque entièrement sur la conséquence. Cela suggère que la taille réduite du modèle (3B) le rend vulnérable aux biais présents dans les exemples fournis ou dans la structure de la consigne.

#### 2. Supériorité de la Pensée par Chaîne (Chain of Thought)
La méthode Chain of Thought (CoT) est la plus performante (40% d'exactitude). En forçant le modèle à décomposer son raisonnement, on réduit les prédictions réflexives (biais de classe) et on améliore la distinction entre les catégories, notamment pour les contradictions qui atteignent une précision de 71%.

#### 3. Difficulté persistante avec la classe "Neutre"
Quelle que soit la méthode, le score F1 pour la catégorie Neutre reste extrêmement bas (0.00 à 0.05). Le modèle semble incapable de saisir l'absence de lien logique, tendant systématiquement à vouloir forcer une relation (soit d'accord, soit d'opposition) entre la prémisse et l'hypothèse.

#### 4. Limites du modèle 3B
Avec une accuracy plafonnant à 40% (contre 33% pour un choix aléatoire), le modèle LLAMA 3.2 3B montre des limites structurelles pour le NLI complexe en français. Il reste peu fiable pour une mise en production nécessitant une analyse logique fine, bien que le raisonnement par étapes (CoT) offre une base d'amélioration prometteuse.

### Variante testés 
Plusieurs méthodes ont étés utilisées pour essayer de corriger les biais du modèle envers certaine étiquette mais sans succès :
- Changement des modèles de prompts:
    - Simplification du langage utilisé dans les prompts.
    - Utilisation de tags xml <label>, <raisonnement>, <hypothèse>, etc. 
    - Changement de l'ordre des exemples pour le few-shot et CoT.
    - Utilisation d'exemples plus simples.
    - Utilisation d'exemples plus explicites.
- Activation / Désactivation de la quantisation 4 bits.